<strong><div dir=rtl >
<font size=4 >2) استخراج ویژگی ها  <font><strong>

<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
در بخش تحلیل احساسات اخبار با استفاده از مدل پارس‌بِرت (ParsBERT) که برای تحلیل احساسات فارسی تنظیم شده است، انجام شده است.

In [ ]:
pip install hazm

<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
داده‌های خبری از فایل‌های CSV مربوط به دسته‌بندی‌های مختلف (اقتصاد، سیاست، صنعت، انرژی و ارز) بارگذاری شده و در یک مجموعه داده واحد به نام all_news ترکیب شده‌اند.
ستون جدیدی به نام category اضافه شده که دسته‌بندی هر خبر را مشخص می‌کند. سپس بخش تیتر خبر و متن خبر با یکدیگر ادغام شده اند.

In [ ]:
import pandas as pd
economy = pd.read_csv('/content/drive/MyDrive/EghtesadOnline-interior-economy-400-403.csv')
politics = pd.read_csv('/content/drive/MyDrive/EghtesadOnline-politics-400-403.csv')
industry = pd.read_csv('/content/drive/MyDrive/EghtesadOnline-industry-400-403.csv')
energy = pd.read_csv('/content/drive/MyDrive/EghtesadOnline-energy-400-403.csv')
curruncy = pd.read_csv('/content/drive/MyDrive/EghtesadOnline-curruncy-400-403.csv')

In [ ]:
economy['category']=['economy' for i in range(len(economy))]
politics['category']=['politics' for i in range(len(politics))]
industry['category']=['industry' for i in range(len(industry))]
curruncy['category']=['curruncy' for i in range(len(curruncy))]
energy['category']=['energy' for i in range(len(energy))]
all_news=pd.concat([economy,politics,industry,curruncy,energy])
all_news.fillna('',inplace=True)

In [ ]:
import numpy as np
all_news['Summary']+=np.array([' ' for i in range(len(all_news))])
all_news['Summary']+=np.array([' ' for i in range(len(all_news))])
all_news['full_news']=all_news['Summary']+all_news['Title']
texts=all_news['full_news'].astype(str).tolist()

<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
با استفاده از کتابخانه Hazm، پیش‌پردازش‌های زیر بر روی متون انجام می‌شود:

- **نرمال‌سازی**: هماهنگ کردن فرم کلمات (مانند اصلاح نیم‌فاصله‌ها).
- **حذف نویسه‌های غیرضروری**: شامل کلمات انگلیسی، اعداد، و علائم نگارشی.
- **توکن‌سازی**: شکستن متن به کلمات جداگانه.
- **حذف کلمات توقف**: کلمات پرتکراری که ارزش معنایی ندارند.
- **ریشه‌یابی (Lemmatization)**: تبدیل کلمات به فرم پایه.
<div>

<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
این مرحله داده‌های متنی را برای مدل آماده می‌کند و بهبود دقت تحلیل احساسات را تضمین می‌کند.
</div>

In [ ]:

# Importing necessary libraries
from hazm import Normalizer, word_tokenize, stopwords_list, Lemmatizer
import re
normalizer=Normalizer()
lemmatizer=Lemmatizer()
processed_news=[]

for doc in texts:
    # Normalize
    doc=normalizer.normalize(doc)
    # Remove English words
    doc=re.sub(r'\b[a-zA-Z]+\b', '', doc)
    # Remove numbers
    doc=re.sub(r'\d+', '', doc)
    # Remove punctuations
    doc=re.sub(r'[^\w\s]', '', doc)
    # Tokenize
    tokens=word_tokenize(doc)
    # Lemmatization and Remove stop words
    tokens=[word for word in tokens if word not in set(stopwords_list())]

    doc=' '.join(tokens)
    processed_news.append(doc)

In [ ]:
all_news['full_news']=processed_news
texts=all_news['full_news'].astype(str).tolist()


<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
مدل پارس‌بِرت و توکنایزر مرتبط، که از پیش برای تحلیل احساسات تنظیم شده‌اند، از مسیر مشخص شده بارگذاری می‌شوند.
مدل برای طبقه‌بندی جملات به سه دسته مثبت، منفی و خنثی تنظیم شده است.

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the tokenizer and model from the directory
model_dir = "/content/drive/MyDrive/fine_tuned_model"
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForSequenceClassification.from_pretrained(model_dir)

# Set the model to evaluation mode
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
متون پیش‌پردازش‌شده به دسته‌های کوچکتر تقسیم و به توکن تبدیل می‌شوند.
مدل برای هر دسته از متون، احتمالات سه کلاس (منفی، مثبت، خنثی) را پیش‌بینی می‌کند.
خروجی‌ها شامل ماتریسی از مقادیر احتمالات است.

In [ ]:
import pandas as pd
import torch
from torch.nn.functional import softmax

# 3. Set device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 4. Tokenize the text data in batches
def tokenize_batch(texts, tokenizer, max_length=128):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# 5. Predict probabilities in batches
batch_size = 16
all_probs = []

model.eval()  # Set model to evaluation mode
with torch.no_grad():
    for i in range(0, len(texts), batch_size):
        # Tokenize the current batch
        batch_texts = texts[i:i + batch_size]
        encoded_batch = tokenize_batch(batch_texts, tokenizer).to(device)

        # Perform predictions
        outputs = model(**encoded_batch)
        logits = outputs.logits  # Raw scores
        probabilities = softmax(logits, dim=1)  # Convert logits to probabilities
        all_probs.extend(probabilities.cpu().numpy())  # Collect probabilities




In [ ]:
df=pd.DataFrame(all_probs,columns=['Negative','Positive','Neutral'])
df

,Negative,Positive,Neutral
0,0.031683,0.536280,0.432037
1,0.030392,0.080310,0.889298
2,0.758777,0.029245,0.211978
3,0.447455,0.207523,0.345023
4,0.337236,0.469807,0.192957
...,...,...,...
51800,0.057186,0.888602,0.054212
51801,0.941298,0.030382,0.028320
51802,0.029500,0.901387,0.069113
51803,0.927091,0.027212,0.045697


<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
نتایج پیش‌بینی مدل در یک جدول جدید ذخیره می‌شود که شامل مقادیر احتمالات احساسات برای هر خبر است.
این داده‌ها به مجموعه داده اصلی ترکیب و در فایل CSV ذخیره می‌شوند.

In [ ]:
all_news = all_news.reset_index(drop=True)
df = df.reset_index(drop=True)
combined = pd.concat([all_news, df], axis=1)

combined

,Title,Summary,Date,category,full_news,Negative,Positive,Neutral
0,خبرهای خوش از افزایش حقوق این گروه از کارگران,نخستین نشست شورای عالی کار در دولت چهاردهم با ...,1403/08/23,economy,شورای کار دولت چهاردهم دستور کار بررسی پیشنویس...,0.031683,0.536280,0.432037
1,قانون رتبه بندی معلمان کی اجرایی می‌شود,نماینده مردم ایذه و باغملک در مجلس گفت: وزارت ...,1403/08/23,economy,نماینده مردم ایذه باغملک مجلس وزارت آموزش پرور...,0.030392,0.080310,0.889298
2,خبرهای مهم از ادغام یا انحلال برخی صندوق‌های ب...,این روزها برای بازنشستگان این نگرانی پیش آمده ...,1403/08/23,economy,روزها بازنشستگان نگرانی پیشآمده حکم جدیدی لایح...,0.758777,0.029245,0.211978
3,حذف صفر از پول ملی چه تاثیری در اقتصاد کشور می...,رئیس کل اسبق بانک مرکزی با تصریح بر این موضوع ...,1403/08/23,economy,رئیسکل اسبق بانک مرکزی تصریح موضوع تورم کاهش ا...,0.447455,0.207523,0.345023
4,خبرهای جدید از تغییر دهک‌ها و مبلغ یارانه,وزیرکار درباره احتمال تغییر در دهک یارانه‌بگیر...,1403/08/23,economy,وزیرکار احتمال دهک یارانهبگیران دهک یارانهها ا...,0.337236,0.469807,0.192957
...,...,...,...,...,...,...,...,...
51800,برنامه جدید برای روستاها,وزیر نیرو گفت: شورای جهادی آب در کشور از یک ما...,1400/08/23,energy,وزیر نیرو شورای جهادی آب کشور ماه فعالشده بنا ...,0.057186,0.888602,0.054212
51801,ذخایر آب تهران با کمبود ۴۱درصدی مواجه است,معاون شرکت تامین و تصفیه آبفای تهران درباره وض...,1400/08/23,energy,معاون شرکت تامین تصفیه آبفای تهران وضعیت آبهای...,0.941298,0.030382,0.028320
51802,قیر رایگان به بورس نمی‌ رود / رونمایی از سامان...,محمدی گفت: سامانه‌ای برای معاملات قیر تهاتری ط...,1400/08/23,energy,محمدی سامانهای معاملات قیر تهاتری طراحیشده ماه...,0.029500,0.901387,0.069113
51803,تبعات تاسیس سد آبی جدید ترکیه برای ایران,محمد جمال ولی سامان با اشاره به تلاش ناکام ایر...,1400/08/23,energy,محمد جمال سامان اشاره تلاش ناکام ایران عراق جل...,0.927091,0.027212,0.045697


In [ ]:
combined.to_csv("/content/drive/MyDrive/sentiment_results_with_probs.csv", index=False)

<div style="text-align: justify; direction: rtl; font-family: 'B Nazanin'; font-size: 15px;">
این بخش از تحقیق، داده‌های خام متنی را به اطلاعات کمی در قالب احساسات (منفی، مثبت، خنثی) تبدیل می‌کند. این خروجی‌ها به عنوان متغیرهای ورودی در مراحل بعدی، شامل مدل‌سازی پیش‌بینی قیمت سهام، استفاده خواهند شد.